# Creación del modelo de chat

En este notebook crearemos un RAG partiendo de una base de documentos intercalados entre webs de dominio accesible de manera gratuita y articulos de ciencia del deporte (también de dominio público).

In [1]:
import numpy as np
import os
import nest_asyncio
import pickle
from sentence_transformers import SentenceTransformer
import faiss
import ollama 
from tavily import TavilyClient
workpath = 'C:/Users/Legion/TFM/Tareas/CalistenIA'
os.chdir(workpath)


### Crear vector store

In [ ]:
def create_vector_store(docspath, vs_path):
    with open(docspath, 'rb') as file:
        all_docs = pickle.load(file)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    embeddings = np.array(model.encode(all_docs))
    dimension = embeddings.shape[1]  # Dimension of the embeddings
    index = faiss.IndexFlatL2(dimension)  # L2 distance index

    # Add embeddings to the index
    index.add(embeddings)

    # Save index to disk (optional)
    faiss.write_index(index, vs_path)
    print('vector store done!')
    
def load_vector_store(path):
    return faiss.read_index(path)

Una vez tengamos el vector store, dependiendo del mensaje del usuario habrá que buscar documentos relevantes a este.

In [ ]:
def retreive_docs_vs(message, docspath, model, index):
    '''
    (!!!) Requiere que el modelo y la vector store estén declaradas de manera global (!!!)
    '''
    query_vector = model.encode([message])[0]  # Single vector
    D, I = index.search(np.array([query_vector]), k=5)  # Search for top-5 closest vectors
    with open(docspath, 'rb') as file:
        all_docs = pickle.load(file)
    # D contains the distances, I contains the indices of closest vectors
    closest_docs = [all_docs[i] for i in I[0]]
    return closest_docs[0]

## Traer llama3.1 a local


In [ ]:
# DESCARGAR LLAMA (USAR UNA VEZ)
ollama.pull('llama3.1')

In [2]:
# Prueba de uso
response = ollama.chat(model='llama3.1', messages=[
   {
     'role': 'user',
     'content': 'Why is the sky blue?',
   },
 ])

print(response['message']['content'])

The sky appears blue to us because of a phenomenon called scattering. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.

This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh who first described the phenomenon. As a result of this scattering, the blue light becomes distributed throughout the atmosphere and reaches our eyes from all directions, giving the sky its blue appearance.

There are some interesting variations on this theme to note:

1.  **At sunrise and sunset**, when sunlight passes through more of the Earth's atmosphere to reach your eyes, the shorter wavelengths are scattered away by the air molecules, leaving mainly longer wavelengths (like reds and oranges) to enter our eyes. This is why these times of day often display hues of orange, pink, or

#### Clasificar mensaje (deporte o no deporte)

In [ ]:
# Crear modelo para respuestas de clasificacion binaria en base a prompting
modelfile='''
FROM llama3.1
SYSTEM You are a binary answerer. You can only answer 'yes' or 'no' to the questions you are receiving. No more details, just 'yes' or 'no'.
'''

ollama.create(model='grader', modelfile=modelfile)

In [ ]:
def message_classifier(message):
    print('--------MESSAGE CLASSIFICATION STARTED---------')
    response_message_class = ollama.chat(model='grader', messages=[
        {
            'role': 'system',
            'content':"You are a classifier. The goal of your answer is to determine if the user's message is related to sports training, or a completely different topic. Answer 'yes' if it could be interpreted as related to sports, 'no' otherwise."
        },
       {
         'role': 'user',
         'content': f''' Here is my message: {message} \n
     
         Is it sports-related? or am I talking about other topic?
         ''',
       },
        {'role': 'assistant',
        'content': 'One word answer: '}
     ])
    answer = response_message_class['message']['content']
    if 'yes' in answer.lower():
        final_answer = True
    elif 'no' in answer.lower():
        final_answer = False
    else:
        print('Error en la clasificacion del mensaje. Respuesta LLM: '+answer)
        final_answer = False
    print('--------MESSAGE CLASSIFICATION DONE---------')
    return final_answer

#### Message not related to sports

In [ ]:
def wrong_topic_message(message):
    print('--------WRONG TOPIC MESSAGE STARTED---------')
    response_message = ollama.chat(model='grader', messages=[
        {
            'role': 'system',
            'content':"You are a sports chatbot. You have been asked for a topic that is not under the scope of sports. In this case, your main goal is to answer politely to the user saying that you can not provide information related to that topic. Suggest asking for something related to calisthenics, saying that you can help in that matter."
        },
       {
         'role': 'user',
         'content': f''' {message} \n
         ''',
       },
        {'role': 'assistant',
        'content': 'Polite rejection of answer: '}
     ])
    answer = response_message['message']['content']    
    return answer

#### Grader documents

In [ ]:
def grade_document_relevance(message, docs):
    print('--------DOCUMENT GRADER STARTED---------')
    response_grade = ollama.chat(model='grader', messages=[
        {
            'role': 'system',
            'content':'Give a binary answer to the question provided in the end of the text'
        },
       {
         'role': 'user',
         'content': f''' I have some documents. I am looking for an answer to my question in this documents. \n
         Here are the documents: \n\n {docs} \n\n
         Here is the question I want to answer with this documents: {message} \n
     
         Can I find the solution to my question in the documents?
         Please answer 'yes' or 'no' only.
         ''',
       },
        {'role': 'assistant',
        'content': 'One word answer:'}
     ])
    
    answer = response_grade['message']['content']
    if 'yes' in answer.lower():
        final_answer = True
    elif 'no' in answer.lower():
        final_answer = False
    else:
        print('Error en la clasificacion del mensaje. Respuesta LLM: '+answer)
        final_answer = False
    print('--------DOCUMENT GRADER DONE---------')
    return final_answer

#### Answer generation

In [ ]:
def create_answer(message, docs):
    print('--------ANSWER CREATION STARTED---------')
    response_answer = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'system',
            'content':"You are an assistant for question-answering tasks. Use the context given to answer the question. If you don't get the answer with the context, just say that you don't know. Use three sentences maximum and keep the answer concise."
        },
       {
         'role': 'user',
         'content': f''' Question: {message} \n
         Context: {docs} \n
         ''',
       },{
           'role': 'assistant',
           'content': 'Answer:',
       }
     ])
    print('--------ANSWER CREATION DONE---------')
    return response_answer['message']['content']


#### Hallucination grader

In [ ]:
def hallucination_grader(docs, answer):
    print('--------HALLUCINATION GRADER STARTED---------')
    response_hall_grade = ollama.chat(model='grader', messages=[
        {
            'role': 'system',
            'content':"You are a grader assesing wether an answer is grounded in / supported by a set of facts. provide de binary score as a single word 'yes' or 'no', and no preamble or explanation."
        },
       {
         'role': 'user',
         'content': f''' Here I have documents: {docs} \n
         Here is the answer I am giving from those facts: {answer} \n
         
         Is my answer based in the facts of the document?
         ''',
       },
        {'role': 'assistant',
        'content': 'One word answer:'}
     ])
    response_bot = response_hall_grade['message']['content']
    if 'yes' in response_bot.lower():
        final_answer = True
    elif 'no' in response_bot.lower():
        final_answer = False
    else:
        print('Error en la clasificacion del mensaje. Respuesta LLM: '+response_bot)
        final_answer = False
    print('--------HALLUCINATION GRADER DONE---------')
    return final_answer

#### Check answer helpful for the message

In [13]:
def check_question_answering(message, answer):
    print('--------QA CHECK STARTED---------')
    response_check_qa = ollama.chat(model='grader', messages=[
        {
            'role': 'system',
            'content':"You are a grader assesing wether an answer is helpful for a previous message. provide de binary score as a single word 'yes' or 'no', and no preamble or explanation."
        },
       {
         'role': 'user',
         'content': f''' Here I have my message: {message} \n
         Here is the answer I am receiving: {answer} \n
         
         Is this answer actually helpful for my message?
         ''',
       },
        {'role': 'assistant',
        'content': 'One word score: '}
     ])
    response_bot = response_check_qa['message']['content']
    if 'yes' in response_bot.lower():
        final_answer = True
    elif 'no' in response_bot.lower():
        final_answer = False
    else:
        print('Error en la clasificacion del mensaje. Respuesta LLM: '+ response_bot)
        final_answer = False
    print('--------QA CHECK DONE---------')
    return final_answer

{'model': 'grader', 'created_at': '2024-08-14T09:22:27.1634162Z', 'message': {'role': 'assistant', 'content': ' {"score": "yes"}'}, 'done_reason': 'stop', 'done': True, 'total_duration': 10659204300, 'load_duration': 7946059000, 'prompt_eval_count': 178, 'prompt_eval_duration': 1318030000, 'eval_count': 7, 'eval_duration': 1368651000}


In [10]:
tavily = TavilyClient(api_key='-------API KEY--------')

web_answer = tavily.search(query=question)
print(web_answer)

def tavily_doc_retrieval(tavily_prompt):
    tavily = TavilyClient(api_key='-------API KEY--------')
    web_answer = tavily.search(query=tavily_prompt)
    doc = ''
    for web_index in range(len(web_answer['results'])):
        doc += '-------------START WEB PAGE------------------'
        doc += '# WEB PAGE ' + str(web_index + 1) + ' \n'
        doc += '## URL: ' + web_answer['results'][web_index]['url'] + ' \n'
        doc += '### TITLE: ' + web_answer['results'][web_index]['title'] + ' \n'
        doc += '#### CONTENT: \n ' web_answer['results'][web_index]['content'] + ' \n'
        doc += '-------------END WEB PAGE------------------'
    return doc

    
def tavily_answering(message):
    if len(message) < 140:
        doc = tavily_doc_retrieval(message)
        websearch_answer = ollama.chat(model='llama3.1', messages=[
            {
                'role': 'system',
                'content':"""You are a content summarizer. You get documents based on webpages search, your goal is to extract the key information from the webpages' document.
                give an answer to the question based in the webpages'document. 
                Provide the URLS in the answer so the user can search more information.
                It is important to state in the answer that this is an answer based in web search, so there could be some mistakes in the given information.
                Encourage the user to verify the information.
                """
            },
           {
             'role': 'user',
             'content': f''' Question: {message} \n
             Webpages' documents: {doc} \n
             ''',
           },{
               'role': 'assistant',
               'content': 'Answer:',
           }
         ])
    else: 
        response_prompt = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'system',
            'content':"""You are a question summarizer. You get a message from the user, your goal is to extract the key information to create a short question that sums up what the user is looking for.
            The question will be used to search on the internet for an answer. Make the question efficient for a proper google search
                """
        },
        {
            'role': 'user',
            'content': f''' Question: {message} \n
            ''',
        },{
            'role': 'assistant',
            'content': 'Summarized One sentence question:',
            }
        ])
        doc = tavily_doc_retrieval(response_prompt['message']['content'])
        websearch_answer = ollama.chat(model='llama3.1', messages=[
            {
                'role': 'system',
                'content':"""You are a content summarizer. You get documents based on webpages search, your goal is to extract the key information from the webpages' document.
                give an answer to the question based in the webpages'document. 
                Provide the URLS in the answer so the user can search more information.
                It is important to state in the answer that this is an answer based in web search, so there could be some mistakes in the given information.
                Encourage the user to verify the information.
                """
            },
           {
             'role': 'user',
             'content': f''' Question: {message} \n
             Webpages' documents: {doc} \n
             ''',
           },{
               'role': 'assistant',
               'content': 'Answer:',
           }
         ])
    print('--------ONLINE ANSWER CREATED---------')
    return websearch_answer['message']['content']

{'query': '¿Cómo hacer una hamburguesa?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Cómo hacer hamburguesas caseras (con imágenes) - wikiHow', 'url': 'https://es.wikihow.com/hacer-hamburguesas-caseras', 'content': 'Cómo hacer hamburguesas caseras. ¡Tira a la basura esas hamburguesas de carne procesada rellena de grasa nada saludable que tienes! ... Para hacer una hamburguesa, compra carne molida de res con un 15 % de grasa y colócala en un tazón. Agrégale cebolla y ajo picados, y mezcla todos los ingredientes con una cuchara. Luego, casca 1 huevo ...', 'score': 0.9995661, 'raw_content': None}, {'title': 'Receta de Hamburguesa Clásica l Fácil y Rápida! - Easy Recetas', 'url': 'https://easyrecetas.com/receta/hamburguesa-clasica/', 'content': 'Cómo hacer hamburguesas clásica. 1. Precaliente la parrilla a 375 grados F (medio-alto). En un tazón grande, agregue la carne. Espolvoree uniformemente con la salsa inglesa, sal de sazonar, ajo en polvo y pimi

## Juntar los pasos en un LLM Chaining 

In [ ]:
def pipeline_qa(user_prompt, model, index):
    sports_related = message_classifier(user_prompt)
    if sports_related:
        related_doc = retreive_docs_vs(user_prompt, './Models/RAG/all_docs.pickle', model, index)
        doc_related_to_prompt = grade_document_relevance(user_prompt, related_doc)
        if doc_related_to_prompt:
            answer_prompt = create_answer(user_prompt, related_doc)
            answer_related_to_docs = hallucination_grader(related_doc, answer_prompt)
            if answer_related_to_docs:
                answers_user_question = check_question_answering(user_prompt, answer_prompt)
                if answers_user_question:
                    user_answer = answer_prompt
                else:
                    user_answer = tavily_answering(user_prompt)
            else:
                user_answer = tavily_answering(user_prompt)
        else:
            user_answer = tavily_answering(user_prompt)
    else:
        user_answer = wrong_topic_message(user_prompt)
        
    return user_answer
